In [1]:
import pyvisa
import time
from typing import Optional
import chipwhisperer as cw
from Crypto.Cipher import AES
from tqdm.notebook import tnrange

# Configure the FPGA target
def configure_target():
    bitstreamfile = './hardware/victims/firmware/cw305_aes/cw305_top.bit'
    target = cw.target(None, cw.targets.CW305, force=True, bsfile=bitstreamfile)
    '''
    bitstreamfile = './hardware/victims/firmware/cw305_aes/cw305_top.bit'
    target = cw.target(None, cw.targets.CW305, fpga_id="100t") # or 
    spi = target.spi_mode()
    spi.erase_chip() # erase full chip
    with open(bitstreamfile, "rb") as f:
        data = list(f.read())
        spi.program(data)
    '''
    target.vccint_set(1.0)
    target.pll.pll_enable_set(True)
    target.pll.pll_outenable_set(False, 0)
    target.pll.pll_outenable_set(True, 1)
    target.pll.pll_outenable_set(False, 2)
    target.pll.pll_outfreq_set(10E6, 1)

    target.clkusbautooff = True
    target.clksleeptime = 1

    return target

# Initialize the target
target = configure_target()

(ChipWhisperer Target WARNING|File CW305.py:345) Verilog defines not found in default location (c:\users\ahmed\chipwhisperer5_64\cw\home\portable\chipwhisperer\software\chipwhisperer/../../hardware/victims/cw305_artixtarget/fpga/common/cw305_defines.v).
Using defines from CW305.py.If this isn't what you want, either add 'slurp=False', or provide defines location in 'defines_files'


In [2]:
# Initialize the key and plaintext generator
ktp = cw.ktp.Basic()

# Lists to store keys and plaintexts
textin = []
keys = []
N = 1000  # Number of traces
traces = []

# Integrate trace capture in the loop
for i in tnrange(N, desc='Capturing traces'):
    key, text = ktp.next()  # Generate the next key and plaintext pair
    textin.append(text)
    keys.append(key)
    
    # Encrypt using the CW305 target
    ciphertext = cw.encrypt(target, text, key)  # Note: capture_trace should be called after setting key and plaintext
    cipher = AES.new(bytes(key), AES.MODE_ECB)  # Initialize cipher for verification
    expected_ciphertext = cipher.encrypt(bytes(text))
    assert list(ciphertext) == list(expected_ciphertext), \
        f"Incorrect encryption result! Got {ciphertext} Exp {list(expected_ciphertext)}"

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

AssertionError: Incorrect encryption result! Got bytearray(b'\xb1\x94\xe9\xb8H9\x15\xe6\xf4x\xab\x0bP\xe0;w') Exp [18, 191, 151, 167, 139, 105, 253, 51, 54, 234, 26, 212, 51, 25, 64, 77]